In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import os
# os.environ['ATTN_BACKEND'] = 'xformers'   # Can be 'flash-attn' or 'xformers', default is 'flash-attn'
os.environ['SPCONV_ALGO'] = 'native'        # Can be 'native' or 'auto', default is 'auto'.
                                            # 'auto' is faster but will do benchmarking at the beginning.
                                            # Recommended to set to 'native' if run only once.

from argparse import ArgumentParser
import imageio
from PIL import Image
from mesh_extraction.trellis.pipelines import TrellisImageTo3DPipeline
from mesh_extraction.trellis.utils import render_utils, postprocessing_utils

[SPARSE] Backend: spconv, Attention: flash_attn
Warp 1.5.0 initialized:
   CUDA Toolkit 12.6, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 3090" (24 GiB, sm_86, mempool enabled)
   Kernel cache:
     /home/ehliang/.cache/warp/1.5.0
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def get_pipeline():
    pipeline = TrellisImageTo3DPipeline.from_pretrained("JeffreyXiang/TRELLIS-image-large")

    return pipeline

pipeline = get_pipeline()

[SPARSE][CONV] spconv algo: native
[ATTENTION] Using backend: flash_attn


Using cache found in /home/ehliang/.cache/torch/hub/facebookresearch_dinov2_main
/home/ehliang/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/ehliang/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/ehliang/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


In [4]:
def generate(source_path, output_path, pipeline, pointcloud_path):

    pipeline.cuda()

    images = []

    frame_names = [
        p for p in os.listdir(source_path)
    ]
    frame_names.sort(key=lambda p: int(os.path.splitext(p)[0].split("_")[1]))

    # Load an image
    for i, image_name in enumerate(frame_names):
        if i % 1 == 0:
            print(image_name)
            images.append(Image.open(os.path.join(source_path, image_name)))


    # Run the pipeline
    output = pipeline.run(
        images,
        # Optional parameters
        seed=1,
        main_path=pointcloud_path,
        # sparse_structure_sampler_params={
        #     "steps": 12,
        #     "cfg_strength": 7.5,
        # },
        # slat_sampler_params={
        #     "steps": 12,
        #     "cfg_strength": 3,
        # },
    )
    

    # Render the outputs
    video = render_utils.render_video(output['gaussian'][0])['color']
    imageio.mimsave("sample_gs.mp4", video, fps=30)
    video = render_utils.render_video(output['mesh'][0])['normal']
    imageio.mimsave("sample_mesh.mp4", video, fps=30)
    # GLB files can be extracted from the output
    glb = postprocessing_utils.to_glb(
        output['gaussian'][0],
        output['mesh'][0],
        # Optional parameters
        simplify=0.1,          # Ratio of triangles to remove in the simplification process
        texture_size=1024,      # Size of the texture used for the GLB
    )
    glb.export(output_path)

In [7]:

source_path = "/store/real/ehliang/r2c2r_blender_data/r2c2r_data/test/StorageFurniture/44781/loop_0/link_3_rgb"
model_path = "outputs/merged_example.glb"
pointcloud_path = "/store/real/ehliang/r2c2r_blender_data/r2c2r_data/test/StorageFurniture/44781/loop_0/link_3.ply"

generate(source_path, model_path, pipeline, pointcloud_path)

rgb_0.png
rgb_1.png
rgb_2.png
rgb_3.png
rgb_4.png
rgb_5.png
rgb_6.png
rgb_7.png
rgb_8.png
rgb_9.png
rgb_10.png
rgb_11.png


Sampling: 100%|██████████| 25/25 [00:05<00:00,  4.41it/s]
Rendering: 300it [00:02, 125.60it/s]
Rendering: 300it [00:02, 104.76it/s]


In [6]:
import torch

model = torch.load("/store/real/ehliang/r2c2r_blender_data/r2c2r_data/test/StorageFurniture/44781/loop_0/link_3_occ_partial_64.pt")

model = model.unsqueeze(dim=0)



torch.Size([1, 64, 64, 64])